# Prediction relevance of article of future year 


### Packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functions    # my own functions which are used in more notebooks
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
import tqdm
import math
import kds
import pickle
from tqdm import tqdm
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
import gensim
print((gensim.__version__))  
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
warnings.filterwarnings('ignore')
from sklearn.metrics import roc_curve, auc, roc_auc_score,classification_report, accuracy_score,precision_score,recall_score
from re import search
from sklearn.ensemble import RandomForestClassifier
from itertools import chain

4.1.2


## Upload cleaned abstracts

In [3]:
df_all = pd.read_csv("outputs/df_sw_tok_low_punc_lemm_v7.csv").rename(columns = {'doi_x':'doi'}).set_index("doi")

In [4]:
len(df_all)

476175

In [5]:
df_all.head(2)

,Unnamed: 0,Year,Month,abstract,cord_uid,journal,license,authors,len,language,abstract_cleaned
doi,,,,,,,,,,,
10.1186/1471-2334-1-6,0,2001.0,7.0,OBJECTIVE: This retrospective chart review des...,ug7v899j,BMC Infect Dis,no-cc,"Madani, Tariq A; Al-Ghamdi, Aisha A",1158,en,retrospective chart review describes epidemiol...
10.1186/rr14,1,2000.0,8.0,Inflammatory diseases of the respiratory tract...,02tnwd4m,Respir Res,no-cc,"Vliet, Albert van der; Eiserich, Jason P; Cros...",718,en,inflammatory disease respiratory tract commonl...


In [6]:
df_all.reset_index()[df_all.reset_index()["doi"]=="10.31539/jks.v4i2.1946"]

,doi,Unnamed: 0,Year,Month,abstract,cord_uid,journal,license,authors,len,language,abstract_cleaned
165078,10.31539/jks.v4i2.1946,220841,2021.0,1.0,Penelitian ini bertujuan untuk mengetahui hubu...,zx7ykdoo,Jurnal Keperawatan Silampari,unk,"Yupartini, Lisnawati Rustiawati Epi Sulastri Tuti",909,en,penelitian ini bertujuan untuk mengetahui hubu...


## Parameters

In [7]:
train_year_of_citations = 2021

# which articles based on year of publication will be selected for training and testing
min_train_year_published = 2019
max_train_year_published = 2020
min_pred_year_published = 2021
max_pred_year_published = 2022

embeddings_from_year = 2020
embeddings_to_year = 2021

classifier = "lr"  # or "rf"

# Add target flag for dataframe with dois

In [8]:
train_df = functions.add_target_opencitatins_marginal(target_year = train_year_of_citations,df = df_all,target_col_name="target")

b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    8576
1    4996
Name: target, dtype: int64


In [9]:
train_df

,Unnamed: 0,Year,Month,abstract,cord_uid,journal,license,authors,len,language,abstract_cleaned,OpenCitations,median_cit,target
0,1,2000.0,8.0,Inflammatory diseases of the respiratory tract...,02tnwd4m,Respir Res,no-cc,"Vliet, Albert van der; Eiserich, Jason P; Cros...",718,en,inflammatory disease respiratory tract commonl...,1.0,1.0,0
1,27,2005.0,3.0,mRNA pseudoknots have a stimulatory function i...,5dk231qs,Nucleic Acids Res,no-cc,"Plant, Ewan P.; Dinman, Jonathan D.",652,en,mrna pseudoknots stimulatory function programm...,1.0,1.0,0
2,29,2005.0,3.0,While it is universally accepted that intact R...,1pq6dkl5,Nucleic Acids Res,no-cc,"Imbeaud, Sandrine; Graudens, Esther; Boulanger...",1020,en,universally accepted intact rna constitutes be...,1.0,1.0,0
3,40,2005.0,11.0,BACKGROUND: Salmonid fishes are among the most...,1r65yam5,BMC Mol Biol,cc-by,"Olsvik, Pål A; Lie, Kai K; Jordal, Ann-Elise O...",891,en,salmonid fish among widely studied model fish ...,2.0,1.0,1
4,60,2006.0,5.0,BACKGROUND: Case definitions have been recogni...,w5fxen70,BMC Public Health,cc-by,"Krause, Gérard; Brodhun, Bonita; Altmann, Dori...",1347,en,case definition recognized important element p...,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,992784,2020.0,10.0,The relationship between SARS-CoV-2 viral load...,lydvs1n1,Nat Commun,cc-by,"Fajnzylber, Jesse; Regan, James; Coxen, Kendyl...",833,en,relationship sars-cov-2 viral load risk diseas...,2.0,1.0,1
13568,992796,2020.0,7.0,Exaggerations of the detrimental impact of rec...,z514v0um,Neuron,no-cc,"Hart, Carl L.",268,en,detrimental impact recreational drug use human...,2.0,1.0,1
13569,992802,2019.0,6.0,Coronaviruses cause respiratory tract infectio...,oea5wvzy,Nat Struct Mol Biol,no-cc,"Tortorici, M. Alejandra; Walls, Alexandra C.; ...",838,en,coronaviruses cause respiratory tract infectio...,4.0,1.0,1
13570,992828,2020.0,5.0,The reemergence of coronavirus prompts the nee...,y6pw87s9,J Biomol Struct Dyn,no-cc,"Adeoye, Akinwunmi O.; Oso, Babatunde Joseph; O...",1163,en,reemergence coronavirus prompt need developmen...,1.0,1.0,0


In [10]:
train_df = train_df[(train_df['Year']<=max_train_year_published) & (train_df['Year']>=min_train_year_published)] 
print(len(train_df))

predict_df = df_all[(df_all['Year']<=max_pred_year_published) & (df_all['Year']>=min_pred_year_published)]
#predict_df = df_all
print(len(predict_df))

9277
281752


In [11]:
### Random sample of 1000 articles 
#predict_df.sample(1000).to_csv("outputs/random_samp_1000_articles.csv")

# BOW + CF

### Features - BOW Binary input matrix

In [12]:
cvec = CountVectorizer(analyzer = "word", tokenizer=lambda txt: txt.split(), 
                       ngram_range=(1,1),
                       binary= True,
                       min_df = 1) 

matrix_bow_train = cvec.fit_transform(train_df['abstract_cleaned'])
tokens_bow_train = cvec.get_feature_names_out()

with open('outputs/classifier/train_'+str(train_year_of_citations)+'/tokens_bow_'+str(train_year_of_citations)+'.data', 'wb') as filehandle:
    pickle.dump(tokens_bow_train, filehandle)
    
matrix_bow_train_pd = pd.DataFrame.sparse.from_spmatrix(matrix_bow_train, columns = tokens_bow_train)
matrix_bow_train_pd = matrix_bow_train_pd[sorted(matrix_bow_train_pd.columns)]
print(matrix_bow_train.shape)

(9277, 36269)


In [13]:
# for predicting we need to have the same features like in training, not new, not less ! 
matrix_bow_test = cvec.fit_transform(predict_df['abstract_cleaned'])
tokens_bow_test = cvec.get_feature_names_out()
matrix_bow_test_pd = pd.DataFrame.sparse.from_spmatrix(matrix_bow_test, columns = tokens_bow_test)
print(len(matrix_bow_test_pd))

not_in_test = np.setdiff1d(tokens_bow_train,tokens_bow_test)
columns_in_both = list(set(tokens_bow_train) & set(tokens_bow_test))
matrix_bow_test_pd_without_new_col = matrix_bow_test_pd[columns_in_both]

for col in tqdm(not_in_test): 
    matrix_bow_test_pd_without_new_col[col] =  np.nan
print(len(matrix_bow_test_pd_without_new_col.columns))

matrix_bow_test_pd = matrix_bow_test_pd_without_new_col.fillna(0)
matrix_bow_test_pd = matrix_bow_test_pd[sorted(matrix_bow_test_pd.columns)]

281752


100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 96.57it/s]


36269


In [14]:
X_train_bow = matrix_bow_train_pd
y_train_bow = train_df.target

#  Classifier

In [15]:
if classifier == "lr":
    cf_bow_splitted_train = LogisticRegression(random_state=0, penalty = "l1",solver = "saga").fit(X_train_bow, y_train_bow)
    pickle.dump(cf_bow_splitted_train, open('outputs/classifier/train_'+str(train_year_of_citations)+'/lreg_bow_'+str(train_year_of_citations)+'.sav', 'wb'))

if classifier == "rf":
    cf_bow_splitted_train = RandomForestClassifier(random_state=0).fit(X_train_bow, y_train_bow)
    pickle.dump(cf_bow_splitted_train, open('outputs/classifier/train_'+str(train_year_of_citations)+'/rf_bow_'+str(train_year_of_citations)+'.sav', 'wb'))

In [16]:
batch_size = 50000
num_samples = matrix_bow_test_pd.shape[0]

y_pred_bow_test = np.empty((0, ))

for start_idx in range(0, num_samples, batch_size):
    print(start_idx)
    end_idx = min(start_idx + batch_size, num_samples)
    batch_rows = matrix_bow_test_pd.iloc[start_idx:end_idx]

    # Predict probabilities for this batch
    batch_proba = cf_bow_splitted_train.predict_proba(batch_rows.values)[:,1]

    # Accumulate predictions
    y_pred_bow_test = np.append(y_pred_bow_test, batch_proba, axis=0)

0
50000
100000
150000
200000
250000


### Articles with highest score

In [17]:
high_score_art_lr_bow = pd.DataFrame(list(zip(list(y_pred_bow_test), list(train_df.abstract.values),list(train_df.abstract_cleaned.values),list(predict_df.index),list(predict_df.Year.values) )),columns =["score","abstract","abstract_cleaned","doi","Year"])
high_score_art_lr_bow = high_score_art_lr_bow.sort_values("score",ascending=False)
high_score_art_lr_bow[["score","doi","Year"]][:5]

,score,doi,Year
789,0.997247,10.1186/s12943-021-01310-0,2021.0
82,0.994012,10.1038/s41467-020-20544-y,2021.0
3339,0.990473,10.1186/s12199-021-00995-5,2021.0
1350,0.989359,10.3324/haematol.2020.262485,2021.0
3080,0.989093,10.1111/psyp.13796,2021.0


In [18]:
len(high_score_art_lr_bow)

9277

### add info 

In [19]:
df_all = df_all.reset_index()

In [20]:
score_df = high_score_art_lr_bow.merge(df_all[["doi","authors","journal"]],on="doi",how="left")

### Add n most important words 


In [21]:
score_df["abstract_cleaned_tok"] = functions.tokenized_column(score_df["abstract_cleaned"])

feature_importance = functions.importance_lr_bow(tokens_bow_train,cf_bow_splitted_train,n=len(tokens_bow_train),odds_ratio = False)
#feature_importance["score_abs"] = abs(feature_importance["score"])
fi_dict = dict(zip(feature_importance.word,feature_importance.score))

n=10
row_list = []
for row in list(range(0,len(score_df.abstract_cleaned_tok))): 
    words_abstract = list(chain.from_iterable(list(score_df.abstract_cleaned_tok[row:row+1])))
    words_abstract_score = dict(((key, fi_dict[key]) for key in words_abstract))
    words_abstract_score_sort = dict(sorted(words_abstract_score.items(), key=lambda item: item[1],reverse=True)) 
    
    top_n = {k: words_abstract_score_sort[k] for k in list(words_abstract_score_sort)[:n]}
    
    #round values in dictionary
    res = dict()
    for key in top_n:  
        res[key] = round(top_n[key], 2)
    row_list.append(res)
    
score_df["top_n_words"] = row_list

In [22]:
score_df.to_csv('outputs/classifier/train_'+str(train_year_of_citations)+'/final_articles_score_table_bow.csv')

In [23]:
pd.set_option('display.max_colwidth', None)
score_df[["doi","score","Year","authors","journal","top_n_words"]][:10]

,doi,score,Year,authors,journal,top_n_words
0,10.1186/s12943-021-01310-0,0.997247,2021.0,"Huang, Xing; Zhang, Gang; Tang, Tianyu; Liang, Tingbo",Mol Cancer,"{'in-silico': 0.78, 'non-structural': 0.56, 'antiviral': 0.45, 'carried': 0.32, 'entire': 0.31, 'best': 0.3, 'towards': 0.27, 'spike': 0.26, 'receptor': 0.24, 'chinese': 0.23}"
1,10.1038/s41467-020-20544-y,0.994012,2021.0,"Mistry, Dina; Litvinova, Maria; Pastore y Piontti, Ana; Chinazzi, Matteo; Fumanelli, Laura; Gomes, Marcelo F. C.; Haque, Syed A.; Liu, Quan-Hui; Mu, Kunpeng; Xiong, Xinyue; Halloran, M. Elizabeth; Longini, Ira M.; Merler, Stefano; Ajelli, Marco; Vespignani, Alessandro",Nat Commun,"{'unfolded': 0.61, 'jnk': 0.31, 'component': 0.29, 'degradation': 0.25, 'support': 0.22, 'type': 0.22, 'virus': 0.21, 'translocation': 0.19, 'chop': 0.17, 'severe': 0.12}"
2,10.1186/s12199-021-00995-5,0.990473,2021.0,"Johnson, Natalie M.; Hoffmann, Aline Rodrigues; Behlen, Jonathan C.; Lau, Carmen; Pendleton, Drew; Harvey, Navada; Shore, Ross; Li, Yixin; Chen, Jingshu; Tian, Yanan; Zhang, Renyi",Environ Health Prev Med,"{'sequenced': 0.89, 'forecasting': 0.77, 'ten': 0.57, 'collection': 0.51, 'recombinant': 0.47, 'detecting': 0.25, 'thousand': 0.22, 'appear': 0.21, 'virus': 0.21, 'scenario': 0.16}"
3,10.3324/haematol.2020.262485,0.989359,2021.0,"Di Buduo, Christian A.; Aguilar, Alicia; Soprano, Paolo M.; Bocconi, Alberto; Miguel, Carolina P.; Mantica, Giovanna; Balduini, Alessandra",Haematologica,"{'allocation': 0.57, 'limitation': 0.48, 'short-term': 0.39, 'appropriately': 0.34, 'comorbidities': 0.28, 'outcome': 0.24, 'make': 0.22, 'score': 0.22, 'covid-19': 0.22, 'necessary': 0.21}"
4,10.1111/psyp.13796,0.989093,2021.0,"Zimmerman, Benjamin; Rypma, Bart; Gratton, Gabriele; Fabiani, Monica",Psychophysiology,"{'chemistry': 0.78, 'humidity': 0.49, 'across': 0.46, 'air': 0.39, 'sensor': 0.35, 'potentially': 0.26, 'covid-19': 0.22, 'movement': 0.21, 'interaction': 0.2, 'reflect': 0.13}"
5,10.3390/bios11080253,0.987647,2021.0,"Taha, Bakr Ahmed; Ali, Norazida; Sapiee, Nurfarhana Mohamad; Fadhel, Mahmoud Muhanad; Mat Yeh, Ros Maria; Bachok, Nur Nadia; Al Mashhadany, Yousif; Arsad, Norhana",Biosensors (Basel),"{'averaged': 0.57, 'greatest': 0.53, 'january': 0.47, 'pollution': 0.42, 'established': 0.4, 'quarantine': 0.39, 'air': 0.39, 'china': 0.33, 'particulate': 0.32, 'approximately': 0.32}"
6,10.1038/s42003-021-02706-w,0.985561,2021.0,"Natu, Vaidehi S.; Rosenke, Mona; Wu, Hua; Querdasi, Francesca R.; Kular, Holly; Lopez-Alvarez, Nancy; Grotheer, Mareike; Berman, Shai; Mezer, Aviv A.; Grill-Spector, Kalanit",Commun Biol,"{'recovered': 0.51, 'igm': 0.49, 'perinatal': 0.46, 'polymerase': 0.42, 'critically': 0.32, 'completed': 0.29, 'potentially': 0.26, 'spike': 0.26, 'antibody': 0.25, 'receptor': 0.24}"
7,10.1186/s13059-021-02363-6,0.985524,2021.0,"Ho, Won Jin; Erbe, Rossin; Danilova, Ludmila; Phyo, Zaw; Bigelow, Emma; Stein-O’Brien, Genevieve; Thomas, Dwayne L.; Charmsaz, Soren; Gross, Nicole; Woolman, Skylar; Cruz, Kayla; Munday, Rebecca M.; Zaidi, Neeha; Armstrong, Todd D.; Sztein, Marcelo B.; Yarchoan, Mark; Thompson, Elizabeth D.; Jaffee, Elizabeth M.; Fertig, Elana J.",Genome Biol,"{'checklist': 0.84, 'urgently': 0.75, 'guiding': 0.69, 'finally': 0.65, 'mostly': 0.52, 'systematic': 0.42, 'ovid': 0.36, 'diagnosing': 0.34, 'pubmed': 0.34, 'probably': 0.33}"
8,10.1371/journal.pone.0258333,0.985388,2021.0,"Velten, Julia; Scholten, Saskia; Brailovskaia, Julia; Margraf, Jürgen",PLoS One,"{'chloroquine': 0.58, 'antiviral': 0.45, 'phenotype': 0.45, 'mixture': 0.4, 'statistically': 0.39, 'ongoing': 0.36, 'best': 0.3, 'criterion': 0.29, 'prospective': 0.29, 'society': 0.28}"
9,10.1038/s41467-021-24885-0,0.984414,2021.0,"Salehinejad, Mohammad Ali; Wischnewski, Miles; Ghanavati, Elham; Mosayebi-Samani, Mohsen; Kuo, Min-Fang; Nitsche, Michael A.",Nat Commun,"{'kinetics': 0.43, 'dataset': 0.37, 'crp': 0.36, 'interpretation': 0.34, 'carried': 0.32, 'rather': 0.31, 'stage': 0.29, 'serial': 0.26, 

# EMBEDDINGS

## Agregating w2v Embeddings + LR

- w2v was trained on all articles from kaggle (with or without target - openctitations) - to the target year of publication

In [24]:
model_w2v = gensim.models.Word2Vec.load("outputs/w2v/w2v_published_between_"+str(embeddings_from_year) + " and "+ str(embeddings_to_year)+".model")

In [25]:
train_df["abstract_tokenized"] = functions.tokenized_column(train_df.reset_index()["abstract_cleaned"])
df_X_train_avg = functions.transform_to_document_vector(text_col_tokenized = train_df.reset_index().abstract_tokenized,model = model_w2v,index_col_list = list(train_df.index),agg_func = "avg").fillna(0)

predict_df["abstract_tokenized"] = functions.tokenized_column(predict_df.reset_index()["abstract_cleaned"])
df_X_predict_avg = functions.transform_to_document_vector(text_col_tokenized = predict_df.reset_index().abstract_tokenized,model = model_w2v,index_col_list = list(predict_df.index),agg_func = "avg").fillna(0)

100%|██████████████████████████████████████████████████████████████████████████| 281752/281752 [48:17<00:00, 97.24it/s]


## LR

In [26]:
X_train_w2v_avg = df_X_train_avg.reset_index().rename({"index":"doi"},axis="columns")
X_test_w2v_avg = df_X_predict_avg.reset_index().rename({"index":"doi"},axis="columns")
y_train_w2v = y_train_bow

In [27]:
X_test_w2v_avg = X_test_w2v_avg.set_index("doi")

In [28]:
X_train_w2v_avg = X_train_w2v_avg.set_index("doi")

In [29]:
if classifier == "lr":
    cf_w2v_splitted_train_avg = LogisticRegression(random_state=0,penalty = "l1",solver = "saga").fit(X_train_w2v_avg, y_train_w2v)
    pickle.dump(cf_w2v_splitted_train_avg, open('outputs/classifier/train_'+str(train_year_of_citations)+'/'+'lreg_w2v_avg_'+str(train_year_of_citations)+'.sav', 'wb'))
    
if classifier == "rf":
    cf_w2v_splitted_train_avg = RandomForestClassifier(random_state=0).fit(X_train_w2v_avg, y_train_w2v)
    pickle.dump(cf_w2v_splitted_train_avg, open('outputs/classifier/train_'+str(train_year_of_citations)+'/'+'rf_w2v_avg_'+str(train_year_of_citations)+'.sav', 'wb'))
     
y_pred_w2v_predict_avg = cf_w2v_splitted_train_avg.predict_proba(df_X_predict_avg)[:,1]

### Abstracts with highest score

In [30]:
high_score_art_lr_w2v_avg = pd.DataFrame(list(zip(list(y_pred_w2v_predict_avg), list(predict_df.abstract.values),list(predict_df.abstract_cleaned),list(predict_df.index),list(predict_df.Year.values))),columns =["score","abstract","abstract_cleaned","doi","Year"])
high_score_art_lr_w2v_avg = high_score_art_lr_w2v_avg.sort_values("score",ascending=False)
#high_score_art_lr_w2v_avg.to_csv('outputs/classifier/train_'+str(train_year_of_citations)+'/'+'high_score_art_lr_w2v_avg.csv')
high_score_art_lr_w2v_avg[["score","doi","Year"]][:10]

,score,doi,Year
77904,0.874052,10.31539/jks.v4i2.1946,2021.0
105453,0.868661,10.31234/osf.io/38c7u,2021.0
106378,0.862832,10.24167/psidim.v20i2.3507,2021.0
75380,0.861113,10.30651/blc.v18i1.5446,2021.0
187186,0.855088,10.1093/infdis/jiab355,2021.0
96547,0.852730,10.24912/jmishumsen.v5i2.11786.2021,2021.0
69471,0.851538,10.35914/tomaega.v4i1.474,2021.0
95350,0.847378,10.1101/2021.12.25.21268301,2021.0
82162,0.846596,10.1142/s0219477521500528,2021.0
71408,0.845730,10.22435/hsji.v12i1.4879,2021.0


In [31]:
score_df = high_score_art_lr_w2v_avg.merge(df_all[["doi","authors","journal"]],on="doi",how="left")

In [32]:
score_df

,score,abstract,abstract_cleaned,doi,Year,authors,journal
0,0.874052,"Penelitian ini bertujuan untuk mengetahui hubungan stres kerja tenaga kesehatan dengan perilaku caring di Puskesmas Kota Serang. Jenis penelitian ini adalah survey analitik dengan desain cross-sectional. Hasil penelitian didapatkan 43,2% tenaga kesehatan dengan stres sedang dan 29,1% tenaga kesehatan kurang caring. Hasil analisis bivariat menunjukkan bahwa nilai (p = 0.00). Simpulan, terdapat hubungan antara stres kerja tenaga kesehatan dengan perilaku caring di Puskesmas di Kota Serang Tahun 2020. Kata Kunci: Caring, Pandemi COVID-19, Stres Kerja, Tenaga Kesehatan This study aims to determine the relationship between work stress of health workers and caring behavior at the Serang City Health Center. This type of research is an analytic survey with a cross-sectional design. The results showed that 43.2% of health workers with moderate stress and 29.1% of health workers lacked caring. The results of the bivariate analysis showed that the value (p = 0.00). In conclusion there is a relationship between the work stress of health workers and caring behavior at the Puskesmas in Serang City in 2020. Keywords: Caring, COVID-19 Pandemic, Work Stress, Health Workers",penelitian ini bertujuan untuk mengetahui hubungan kerja tenaga kesehatan dengan perilaku caring di puskesmas kota serang jenis penelitian ini adalah survey dengan desain cross-sectional hasil penelitian didapatkan tenaga kesehatan dengan sedang dan tenaga kesehatan kurang caring hasil analisis menunjukkan bahwa nilai p terdapat hubungan antara kerja tenaga kesehatan dengan perilaku caring di puskesmas di kota serang tahun 2020 kata kunci caring pandemi covid-19 kerja tenaga kesehatan study aim determine relationship work stress health worker caring behavior serang city health center type research analytic survey cross-sectional design result showed health worker moderate stress health worker lacked caring result bivariate analysis showed value p conclusion relationship work stress health worker caring behavior puskesmas serang city 2020 keywords caring covid-19 pandemic work stress health worker,10.31539/jks.v4i2.1946,2021.0,"Yupartini, Lisnawati Rustiawati Epi Sulastri Tuti",Jurnal Keperawatan Silampari
1,0.868661,Neuropsychiatric symptoms caused by COVID-19,neuropsychiatric symptom caused covid-19,10.31234/osf.io/38c7u,2021.0,"Inasaridze, Ketevan",NaN
2,0.862832,Studi ini bertujuan untuk mengukur prevalensi dan asosiasi antara depresi kecemasan stres dan kualitas tidur mahasiswa selama pandemi COVID-19. Tingkat depresi kecemasan dan stres diukur menggunakan DASS-42 versi Indonesia dan kualitas tidur diukur menggunakan PSQI versi Indonesia. Pengambilan data dilakukan secara daring. Sebanyak 271 partisipan dari berbagai program studi menjadi partisipan pada studi ini terdiri dari 220 perempuan dan 51 laki- laki (M=20.11 SD=2.25). Hasil studi ini menunjukkan sebanyak 43.91% partisipan mengalami depresi 69.74% cemas 43.17% stres dari tingkat ringan sampai berat serta 92.25% partisipan memiliki kualitas tidur yang buruk. Analisis korelasi Pearson menunjukkan ada korelasi antara kualitas tidur dengan depresi (r= .313 p .01) kecemasan (r= .433 p .01) dan stres (r= .383 p .01). Analisis regresi sederhana menunjukkan bahwa kualitas tidur secara signifikan mampu memprediksi depresi sebesar 9.8% kecemasan 18.8% dan stres 14.7%. Semakin buruk kualitas tidur maka semakin tinggi tingkat depresi kecemasan dan stres individu. Sebagian besar partisipan mengalami kecemasan dari tingkat ringan sampai berat serta kualitas tidur secara signifikan mampu memprediksi depresi kecemasan dan stres selama pandemi. Kata kunci: depresi kecemasan stres kualitas tidur mahasiswa pandemi COVID-19 The study aims to assess prevalence of depression anxiety stress and sleep quality during the COVID-19 of university student as well as to examine the relationship among those factors. The researcher conducted completely anonymous web-based su

In [33]:
score_df["len"] = score_df['abstract_cleaned'].str.len()

In [34]:
score_df = score_df.sort_values("score",ascending=False)

### Add n most important words 

In [35]:
n=10

In [36]:
score_df["abstract_cleaned_tok"] = functions.tokenized_column(score_df["abstract_cleaned"])

feature_importance = functions.score_of_word(model_w2v,cf_w2v_splitted_train_avg).sort_values(by = ["score"], ascending=False)
fi_dict = dict(zip(feature_importance.word,feature_importance.score))

row_list = []
for row in tqdm(list(range(0,len(score_df.abstract_cleaned_tok)))): 
    words_abstract = list(chain.from_iterable(list(score_df.abstract_cleaned_tok[row:row+1])))
    
    # potrebuji predfiltrovat jen ta slova z abstraktu pro ktera mam FI
    words_abstract = list(set(fi_dict.keys()).intersection(words_abstract))
    
    words_abstract_score = dict(((key, fi_dict[key]) for key in words_abstract))
    words_abstract_score_sort = dict(sorted(words_abstract_score.items(), key=lambda item: item[1],reverse=True)) 
    
    top_n = {k: words_abstract_score_sort[k] for k in list(words_abstract_score_sort)[:n]}
    
    #round values in dictionary
    res = dict()
    for key in top_n:  
        res[key] = round(top_n[key], 2)
    row_list.append(res)
    
score_df["top_n_words"] = row_list

100%|████████████████████████████████████████████████████████████████████████| 281752/281752 [1:12:52<00:00, 64.43it/s]


In [37]:
keywords = ['menunjukkan', 'bahwa', 'pada', 'kasus'] # uniflter indonesia articles

# Funkce pro filtrování
def filter_keywords(row):
    top_n_words = row['top_n_words']
    for keyword in keywords:
        if keyword in top_n_words:
            return False
    return True

# Aplikace filtru
score_df = score_df[score_df.apply(filter_keywords, axis=1)]

In [38]:
pd.set_option('display.max_colwidth', None)
score_df[["doi","score","Year","authors","journal","top_n_words"]][:10]

,doi,score,Year,authors,journal,top_n_words
1,10.31234/osf.io/38c7u,0.868661,2021.0,"Inasaridze, Ketevan",NaN,"{'symptom': 0.95, 'neuropsychiatric': 0.93, 'covid-19': 0.87, 'caused': 0.53}"
4,10.1093/infdis/jiab355,0.855088,2021.0,"Widera, Marek; Wilhelm, Alexander; Hoehl, Sebastian; Pallas, Christiane; Kohmer, Niko; Wolf, Timo; Rabenau, Holger F; Corman, Victor M; Drosten, Christian; Vehreschild, Maria J G T; Goetsch, Udo; Gottschalk, Rene; Ciesek, Sandra",J Infect Dis,"{'imdevimab': 1.0, 'casirivimab': 1.0, 'pseudoviruses': 1.0, 'zeta': 0.99, 'sars-cov-2': 0.98, 'bamlanivimab': 0.98, 'b117': 0.97, 'b1351': 0.96, 'beta': 0.95, 'alpha': 0.92}"
7,10.1101/2021.12.25.21268301,0.847378,2021.0,"Kim, D.; Jo, J.; Lim, J.-S.; Ryu, S.",NaN,"{'omicron': 1.0, 'b11529': 0.99, 'sars-cov-2': 0.98, 'credible': 0.97, 'transmission': 0.96, 'outbreak': 0.93, 'variant': 0.92, 'reproduction': 0.91, 'serial': 0.89, 'interval': 0.85}"
8,10.1142/s0219477521500528,0.846596,2021.0,"Wang, Jian Shao Wei Yan Yan Kim Junseok",Fluctuation and Noise Letters,"{'wuhan': 1.0, 'multifractal': 1.0, 'hubei': 1.0, '“wuhan': 0.99, 'city': 0.98, 'cross-correlations': 0.98, 'china': 0.97, 'coronavirus': 0.96, 'detrended': 0.93, 'outbreak': 0.93}"
11,10.3390/pediatric13010013,0.844665,2021.0,"Caselli, Désirée; Aricò, Maurizio",Pediatr Rep,"{'raging': 0.99, 'covid-19': 0.87, 'pandemic': 0.77, 'still': 0.36}"
12,10.3390/nu14030693,0.844053,2022.0,"Zampelas, Antonis",Nutrients,"{'sars-cov-2': 0.98, 'coronavirus': 0.96, 'outbreak': 0.93, 'covid-19': 0.87, 'disease-2019': 0.76, 'recent': 0.75, '2019': 0.69, 'cause': 0.09}"
15,10.33087/jiubj.v21i3.1725,0.836698,2021.0,"Sari Mila, Triana Putri Miko Eka Daryanto Daryanto",Jurnal Ilmiah Universitas Batanghari Jambi,"{'oleh': 1.0, 'atau': 1.0, 'kesehatan': 1.0, 'adalah': 1.0, 'dengan': 1.0, 'dari': 1.0, 'selama': 1.0, 'ini': 1.0, 'penelitian': 1.0, 'hasil': 1.0}"
17,10.1093/jtm/taac037,0.831371,2022.0,"Liu, Ying; Rocklöv, Joacim",J. travel med,"{'omicron': 1.0, 'transmissibility': 0.99, 'delta': 0.98, 'variant': 0.92, 'reproduction': 0.91, 'average': 0.8, 'higher': 0.79, 'effective': 0.77, 'basic': 0.7, 'time': 0.59}"
19,10.3390/jcm11051336,0.830971,2022.0,"Mallat, Jihad",J Clin Med,"{'sar-cov-2': 1.0, 'scale': 0.98, 'coronavirus-2': 0.97, 'unparalleled': 0.96, 'coronavirus': 0.96, 'severe': 0.95, 'covid-19': 0.87, 'world': 0.86, 'global': 0.8, 'pandemic': 0.77}"
20,10.1111/pai.13661,0.826867,2021.0,"Yilmaz Topal, Ozge; Metin, Ayse; Kulhas Celik, İlknur; Metbulut, Azize Pinar; Alim Aydin, Selma; Kanik Yuksek, Saliha; Ozkaya Parlakay, Aslinur",Pediatr Allergy Immunol,"{'sars-cov-2': 0.98, 'globe': 0.98, 'spread': 0.97, 'rapidly': 0.96, 'coronavirus': 0.96, 'severe': 0.95, 'novel': 0.91, 'covid-19': 0.87, 'world': 0.86, '2020': 0.8}"


In [39]:
score_df.to_csv('outputs/classifier/train_'+str(train_year_of_citations)+'/final_articles_score_table_w2v.csv')